# Ejercicio 1:

Si le pedimos a Julia la representación en binario de 0.1 (flotante de 64-bits), obtenemos:

In [2]:
bits(0.1)

"0011111110111001100110011001100110011001100110011001100110011010"

Si hacemos la división larga del binario 1 (1 en decimal) entre 1010 (10 en decimal), obtenemos que el 0.1 en binario no tiene una representación exacta y es de la forma: 00011001100110011001100... hasta infinito.

Por lo tanto, no importa cuantos bits puedas almacenar en tu computadora, nunca podrás expresar 0.1 exactamente en binario.

Los números flotantes que usa Julia(y casi todos los lenguajes de programación) son representados con un número fijo de bits. En los punto-flotante de doble precisión, por ejemplo, se utilizan 53 bits, por lo que la representación infinita de un número es redondeada a 53 cifras significantes.

En el caso de 0.1, escencialmente, el número se divide en dos partes: el significando y el exponente.
0.1 seria representado en decimal como $1*10^{-1}$.
La computadora, en lugar de almacenarlo como $x10^y$, los almacena como $a2^b$.

Para 0.1, según la convención IEEE para flotantes dobles, el primer bit(en éste caso 0) indica el signo, por lo tanto un 0 indica que nuestro número es positivo.
La segunda parte de 0.1 en flotante(los siguientes 11 bits: 01111111011), almacena el exponente del número.
Los bits restantes son para el significando del número.

# Ejercicio 2:

Implementamos la siguiente función recursiva para encontrar el machine epsilon:

In [39]:
function machineepsilon(x,eps=1.0)
    """Encuentra el machine epsilon de un 
    número flotante determinado, recibe como
    argumentos el flotante."""
    if x+ (eps/2) > x
        return machineepsilon(x,(eps/2)) 
        end
    return eps 
    end


machineepsilon (generic function with 2 methods)

In [38]:
machineepsilon(1.0),eps(1.0)

(2.220446049250313e-16,2.220446049250313e-16)

In [36]:
machineepsilon(10.0),eps(10.0)

(1.7763568394002505e-15,1.7763568394002505e-15)

# Ejercicio 3:

In [3]:
function derivada1(f,xo,h)
    """Obtiene una aproximación a la derivada de 
    f(x) alrededor de x_0"""
    @assert h>0 
    operador= (f(xo+h)-f(xo))/h
    return operador
end

derivada1 (generic function with 1 method)

In [6]:
function xsquared(y)
    return y*y
end

xsquared (generic function with 1 method)

In [23]:
function polejemp(y)
    return 10y^3+5y^2
end

polejemp (generic function with 1 method)

In [25]:
function polinomio2(y)
    poli=-10y^2+19
    return poli
end

polinomio2 (generic function with 1 method)

In [26]:
function lineal(y)
    return 5y
end

lineal (generic function with 1 method)

In [13]:
derivada1(lineal,7,1)

5.0

In [27]:
derivada1(lineal,7,0.1)

5.0

In [39]:
derivada1(polinomio2,5,1)

-110.0

In [40]:
derivada1(polinomio2,5,1e-14)

-99.47598300641403

In [44]:
derivada1(exp,2,1)

12.696480824257018

In [45]:
derivada1(exp,2,1e-14)

7.5495165674510645

In [43]:
derivada1(polejemp,2,1)

215.0

In [31]:
derivada1(polejemp,2,1e-14)

142.10854715202004

Observamos que la fórmula únicamente es exacta para funciones lineales.
Podríamos calcular el Error de aproximación usando la expansión de Taylor alrededor de x:

$$f(x+h)=f(x)+hf'(x)+\frac{h^2f''(\epsilon)}{2}   :\epsilon\in(x,x+h)$$
De lo que se obtiene:
$$f'(x)=\frac{f(x+h)-f(x)}{h}-\frac{hf''(\epsilon)}{2}$$
Observamos entonces, usando notación O grande, que el error es de tiempo $h$, osea, $O(h)$
Esto sugiere que $h$ debe ser tan pequeña como sea posible si queremos reducir el error, pero observamos que como el término de la diferencia es dividido por $h$, hacer $h$ muy pequeño equivale a multiplicar el numerador de la diferencia por un número muy grande. 
Ésto implica que cualquier error de redondeo en el numerador se amplificará y entre más pequeño $h$ éste error aumentará.

Por lo tanto, tenemos que encontrar una $h$ que minimize el error total, compuesto por el error del residuo de Taylor(que disminuye con $h$ chica), y el error de redondeo producto de la aritmética de los punto flotante(que aumenta con $h$ chica).


# Ejercicio 4:

In [32]:
function derivada2(f,xo,h)
    """Obtiene una aproximación a la derivada de 
    f(x) alrededor de x_0"""
    @assert h>0 
    operador= (f(xo+h)-f(xo-h))/(2h)
    return operador
end

derivada2 (generic function with 1 method)

In [36]:
derivada2(lineal,7,1)

5.0

In [37]:
derivada2(lineal,7,0.1)

5.0

In [41]:
derivada2(polinomio2,5,1)

-100.0

In [42]:
derivada2(polinomio2,5,1e-14)

-98.05489753489383

In [46]:
derivada2(exp,2,1)

8.683627547364312

In [47]:
derivada2(exp,2,1e-14)

7.460698725481052

In [48]:
derivada2(polejemp,2,1)

150.0

In [49]:
derivada1(polejemp,2,1e-14)

142.10854715202004

Después de probar los mismos casos que para derivada1, observamos que derivada 2 aproxima mejor los casos aunque $h$ sea grande(1 en éste caso).
Realizamos la expansión de Taylor para ver a que se debe:
$$f(x+h)=f(x)+hf'(x)+\frac{h^2f''(\epsilon)}{2}+\frac{h^3f'''(\epsilon_{1})}{6}:\epsilon_{1}\in(x,x+h)$$
$$f(x-h)=f(x)+hf'(x)+\frac{h^2f''(\epsilon)}{2}-\frac{h^3f'''(\epsilon_{2})}{6}:\epsilon_{2}\in(x-h,x)$$
Por lo tanto:
$$f'(x)=\frac{f(x+h)-f(x-h)}{2h}-\frac{h^2[f'''(\epsilon_{1})+f'''(\epsilon_{2})]}{12}$$
Con lo que se observa que el residuo es ahora de tiempo $h^2$ o $O(h^2)$
Entonces podemos concluir, que a $h$ pequeña, el residuo disminuye más que el error causado por el redondeo del numerador de la diferencia, por lo que obtenemos aproximaciones más exactas.

### Referencias:
Nocedal, Jorge and Stephen J. wright.
1999.
Numerical Optimization.
Springer-
Verlag New York, Inc.